# This is to demo GenAI prompt in FSI

## Financial Analysis Generation by Template

Sell side analyst needs to write analysis report regularly, and there are some template that they use. following is a sample report on Apple Q4 2020 about its quarterly financial report. 

```
Apple Q1 2021 Results Exceed Expectations; Raising FVE to $145

Financial Summary:
Apple reported stellar fiscal Q1 2021 results with revenue of $111.4 billion, up 21% year-over-year, easily beating consensus estimates. All product categories saw double-digit growth led by iPhone (up 17% to $65.6B), Services (up 24% to $15.8B), and Wearables/Home/Accessories (up 30% to $13.0B).

Recommendation:
We reiterate our Buy rating on Apple shares. The stock is currently trading at $137, providing an upside of over 15% from current levels based on our revised $145 fair value estimate.

Risk Factors:
- Potential supply chain disruptions if COVID-19 situation worsens
- Intense competition, especially in 5G smartphones
- Regulatory scrutiny around App Store policies

In [1]:
from IPython.display import HTML

pdf_path = '../reports/Apple-FY21-Q1-Financial-Statement.pdf'
html = f'<iframe src="{pdf_path}" width="100%" height="600px"></iframe>'
HTML(html)


/opt/conda/lib/python3.10/site-packages/IPython/core/display.py:431: UserWarning: Consider using IPython.display.IFrame instead
  warnings.warn("Consider using IPython.display.IFrame instead")


In [15]:
pip install pdfplumber

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.9/57.9 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 70.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 88.9 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [17]:
import pdfplumber

# Open the PDF file
with pdfplumber.open('../reports/Apple-FY21-Q1-Financial-Statement.pdf') as pdf:
    # Initialize an empty string to store the text
    financial_report_text = ''

    # Loop through each page and extract the text
    for page in pdf.pages:
        financial_report_text += page.extract_text()

# Print the extracted text
print(financial_report_text)

Apple Inc.
CONDENSED CONSOLIDATED STATEMENTS OF OPERATIONS (Unaudited)
(In millions, except number of shares which are reflected in thousands and per share amounts)
Three Months Ended
December 26, December 28,
2020 2019
Net sales:
Products $ 95,678 $ 79,104
Services 15,761 12,715
Total net sales (1) 111,439 91,819
Cost of sales:
Products 62,130 52,075
Services 4,981 4,527
Total cost of sales 67,111 56,602
Gross margin 44,328 35,217
Operating expenses:
Research and development 5,163 4,451
Selling, general and administrative 5,631 5,197
Total operating expenses 10,794 9,648
Operating income 33,534 25,569
Other income/(expense), net 45 349
Income before provision for income taxes 33,579 25,918
Provision for income taxes 4,824 3,682
Net income $ 28,755 $ 22,236
Earnings per share:
Basic $ 1.70 $ 1.26
Diluted $ 1.68 $ 1.25
Shares used in computing earnings per share:
Basic 16,935,119 17,660,160
Diluted 17,113,688 17,818,417
(1) Net sales by reportable segment:
Americas $ 46,310 $ 41,367
Eur

GenAI might be able to help with prompt below.

In [20]:
import json
import boto3
newline, bold, unbold = '\n', '\033[1m', '\033[0m'
endpoint_name = 'jumpstart-dft-hf-llm-mistral-7b-ins-20240621-021805'
model_name='huggingface-llm-mistral-7b-instruct-20240621-021806'
def query_endpoint(payload):
    client = boto3.client('runtime.sagemaker')
    response = client.invoke_endpoint(
        EndpointName=endpoint_name, 
        ContentType='application/json', 
        Body=json.dumps(payload).encode('utf-8'),
        InferenceComponentName=model_name
    )
    model_predictions = json.loads(response['Body'].read())
    generated_text = model_predictions[0]['generated_text']
    print (
        f"Generated Text: {bold}{generated_text}{unbold}{newline}")
    # print (
    #     f"Input Text: {payload['inputs']}{newline}"
    #     f"Generated Text: {bold}{generated_text}{unbold}{newline}")

In [32]:
prompt = f"""
You are a financial analyst tasked with analyzing a company's financial report and generating an analyst report. The financial report is provided in PDF format.

Instructions:
1. Carefully read and analyze the PDF financial report, paying close attention to key financial statements, ratios, and other relevant information.

<financial_report>
{financial_report_text}
</financial_report>

2. Generate an analyst report with the following sections:

Financial Report Summary:
- Provide a concise summary of the company's financial performance, highlighting key metrics, trends, and notable figures from the financial statements.
- Discuss the company's revenue, profitability, cash flow, and balance sheet strength.

Buy/Sell Recommendation:
- Based on your analysis, provide a clear recommendation on whether an investor should buy, sell, or hold the company's stock.
- You estimated fair value (FVE), compare it with current apple share price is $130.
- Support your recommendation with specific evidence and reasoning from the financial report analysis.

Risk Alert/Warning:
- Identify and discuss any significant risks, concerns, or potential red flags that emerged from your analysis of the financial report.
- Explain the potential impact of these risks on the company's future performance and valuation.

3. Ensure that your analyst report is well-structured, easy to understand, and provides actionable insights for investors and stakeholders.

4. Double-check your analysis and recommendations for accuracy and consistency with the information provided in the financial report.

Please let me know if you need any clarification or have additional requirements for the analyst report.

"""

payload = {
    "inputs": prompt,
    "parameters":{
        "max_new_tokens": 2000,
        "return_full_text": False,
        "do_sample": True,
        "top_k":10
        }
}


In [33]:
query_endpoint(payload)

Generated Text: ---

**Analyst Report**

**Apple Inc. (AAPL): Q4 2020 Financial Analysis**

**Financial Report Summary:**

Apple Inc. reported strong financial results for the quarter ended December 26, 2020. The company generated net sales of $111.4 billion, a notable increase of 21.2% year-over-year. Apple's profitability remains robust, as evidenced by its operating income of $33.6 billion, a growth of 31.2% compared to Q4 2019.

Key revenue contributors for Apple include its Products segment, which accounted for 86.6% of the total net sales, generating $95.7 billion. The Services segment reported sales of $15.8 billion, a 16.3% increase YoY, indicating a continued focus on expanding its services business.

The cash flow statement reveals that Apple generated $38.8 billion in cash from operating activities, up from $30.5 billion in the same quarter last year. The company's cash position improved, with a cash balance of $37.7 billion, compared to $41.7 billion in the previous quarter